# Grounding a list of metabolites

In [1]:
import re
from gilda import ground

INFO: [2020-12-17 09:52:51] /Users/ben/Dropbox/postdoc/darpa/src/deft/adeft/recognize.py - OneShotRecognizer not available. Extension module for AlignmentBasedScorer is missing


We define some basic functions to load the strings and run Gilda on them. We also define a function to print grounding stats and print out any ungrounded strings.

In [2]:
def load_texts():
    with open('plasmax_name_to_kegg.txt') as fh:
        texts = [l.strip().split(',')[0] for l in fh.readlines()][1:]
    return sorted(set(texts))

def ground_texts(texts, grounding_fun):
    return {text: grounding_fun(text) for text in texts}

def print_grounding_stats(groundings):
    grounded = [t for t, g in groundings.items() if g]
    ungrounded = [t for t, g in groundings.items() if not g]
    num_texts = len(groundings)
    print('Grounded: %d/%d (%.2f%%)' % (len(grounded), num_texts, 100*len(grounded)/num_texts))
    print(ungrounded)

First we try running Gilda without any modifications and see what happens

In [3]:
texts = load_texts()
results = ground_texts(texts, ground)
print_grounding_stats(results)

Grounded: 163/230 (70.87%)
['2-aminomuconicacid', '2-hg', '2/3-phosphoglycerate', '4-pyridoxicacid', '5-phosphoribosyl-1-pyrophosphate', '6-phosphogluconate', 'aconitate', 'akg', 'argininosuccinate', 'ascorbicacid', 'carbamoyl_phosphate', 'carbamoylaspartate', 'carbamoylphosphate', 'cmp-acetylneuraminicacid', 'cysteicacid', 'dihydroacetonephosphate', 'dihydroxyacetonephosphate', 'fructose-16-bisphosphate', 'fructose1-6-bisphosphate', 'fructose1_6-biphosphate', 'glucosamine-6-phosphate', 'glucosamine6-phosphate', 'glutathioneoxidized', 'hydroxyphenyllacticacid', 'indole-3-lacticacid', 'isethionicacid', 'kiv', 'kmv+kic', 'kynurenic_acid', 'kynurenicacid', 'lactoylgsh', 'linoleicacid', 'mannitol/sorbitol', 'methioninesulfoxide', 'methyltryptophan', 'mevalonicacid', 'mevalonicacid5-pyrophosphate', 'myristic_acid', 'myristicacid', 'n-acetylglutamate', 'n-acetylneuramicacid', 'n-methylnicotinamide(nmnm)', 'nicotinamide/picolinamide', 'nicotinamidemononucleotide(nmn)', 'oleic_acid', 'oleicaci

It looks like 71% was grounded. Here is one example, each result is a list of ScoredMatch objects that each contain a Term and some metadata. The grounding is included in the Term. Matches are sorted by decreasing score with the highest scoring match on top.

In [4]:
print(results['lactate'])
print(results['lactate'][0].term.db, results['lactate'][0].term.id)

[ScoredMatch(Term(lactate,lactate,CHEBI,CHEBI:24996,lactate,assertion,famplex,None),1.0,Match(query=lactate,ref=lactate,exact=True,space_mismatch=False,dash_mismatches={},cap_combos=[]))]
CHEBI CHEBI:24996


Upon examination, the entries in the ungrounded list have some patterns of issues that can be fixed with some preprocessing in the `preprocess_text` function. We can then define and use `ground_preprocess` which preprocesses each text before grounding it with Gilda.

In [5]:
typos = {
    'stereamide': 'stearamide',
    'staericacid': 'stearicacid',
}

def preprocess_text(text):
    if text in typos:
        text = typos[text]
    # Example: nicotinamidemononucleotide(nmn)
    text = re.sub('(\([a-zA-Z]+\))$', '', text)
    # Example: palmitic_acid
    text = text.replace('_', ' ')
    # Example: pipecolicacid
    suffixes = ['acid', 'mononucleotide']
    for suffix in suffixes:
        text = re.sub('([^ ])(%s)$' % suffix, '\\1 %s' % suffix, text)
    # Example: nicotinamide/picolinamide
    if '/' in text:
        text = text.split('/')[0]
    return text
        
def ground_preprocess(text):
    text = preprocess_text(text)
    return ground(text)

results = ground_texts(texts, ground_preprocess)
print_grounding_stats(results)

Grounded: 195/230 (84.78%)
['2-hg', '2/3-phosphoglycerate', '5-phosphoribosyl-1-pyrophosphate', '6-phosphogluconate', 'aconitate', 'akg', 'argininosuccinate', 'carbamoylaspartate', 'carbamoylphosphate', 'dihydroacetonephosphate', 'dihydroxyacetonephosphate', 'fructose-16-bisphosphate', 'fructose1-6-bisphosphate', 'fructose1_6-biphosphate', 'glucosamine-6-phosphate', 'glucosamine6-phosphate', 'glutathioneoxidized', 'hydroxyphenyllacticacid', 'kiv', 'kmv+kic', 'lactoylgsh', 'methioninesulfoxide', 'methyltryptophan', 'mevalonicacid5-pyrophosphate', 'n-acetylglutamate', 'n-acetylneuramicacid', 'oleoamide', 'palmitoylcarnitinec16', 'pentose5-phosphates', 'phenolsulphate', 'pyridoxide', 'sedoheptulose7-phosphate', 'seduheptulose7-phosphate', 'succinicglutathione', 'succinylglutathione']


Gilda doesn't have the right synonyms to find groundings for these remaining ungrounded texts.

### Standardizing the results

INDRA offers utilities to map identifiers and standardize names which can be useful in this setting, see https://indra.readthedocs.io/en/latest/modules/ontology/standardize.html.

In [6]:
from indra.ontology.standardize import standardize_name_db_refs

In [7]:
standardize_name_db_refs({results['lactate'][0].term.db: results['lactate'][0].term.id})

INFO: [2020-12-17 09:53:11] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /Users/ben/.indra/bio_ontology/1.5/bio_ontology.pkl


('lactate', {'CHEBI': 'CHEBI:24996', 'CAS': '113-21-3', 'PUBCHEM': '91435'})

We see that the standard name for this entry from CHEBI is `lactate` and we were able to get CAS and PUBCHEM mappings for it.

We can also look at ontological information for the grounded entries via INDRA as follows.

In [9]:
from indra.ontology.bio import bio_ontology
glutamine_term = ground('glutamine')[0].term
children = bio_ontology.get_children(glutamine_term.db, glutamine_term.id)
for c

[('CHEBI', 'CHEBI:144430'),
 ('CHEBI', 'CHEBI:83985'),
 ('CHEBI', 'CHEBI:83080'),
 ('CHEBI', 'CHEBI:26173'),
 ('CHEBI', 'CHEBI:144458'),
 ('CHEBI', 'CHEBI:137239'),
 ('CHEBI', 'CHEBI:16376'),
 ('CHEBI', 'CHEBI:144427'),
 ('CHEBI', 'CHEBI:90444'),
 ('CHEBI', 'CHEBI:138505'),
 ('CHEBI', 'CHEBI:48006'),
 ('CHEBI', 'CHEBI:79289'),
 ('CHEBI', 'CHEBI:72487'),
 ('CHEBI', 'CHEBI:73488'),
 ('CHEBI', 'CHEBI:141431'),
 ('CHEBI', 'CHEBI:88248'),
 ('CHEBI', 'CHEBI:72495'),
 ('CHEBI', 'CHEBI:145321'),
 ('CHEBI', 'CHEBI:3875'),
 ('CHEBI', 'CHEBI:17394'),
 ('CHEBI', 'CHEBI:8087'),
 ('CHEBI', 'CHEBI:24317'),
 ('CHEBI', 'CHEBI:73397'),
 ('CHEBI', 'CHEBI:84058'),
 ('CHEBI', 'CHEBI:27650'),
 ('CHEBI', 'CHEBI:73447'),
 ('CHEBI', 'CHEBI:19111'),
 ('CHEBI', 'CHEBI:84215'),
 ('CHEBI', 'CHEBI:144862'),
 ('CHEBI', 'CHEBI:73846'),
 ('CHEBI', 'CHEBI:141433'),
 ('CHEBI', 'CHEBI:17884'),
 ('CHEBI', 'CHEBI:80662'),
 ('CHEBI', 'CHEBI:73464'),
 ('CHEBI', 'CHEBI:32679'),
 ('CHEBI', 'CHEBI:61521'),
 ('CHEBI', 'CHEBI:762